In [ ]:
import rasterio
import numpy as np
import pandas as pd

# -------------------------------
# 输入路径
# -------------------------------
pv_change_path = r"提取结果_2有无_3无有_4有.tif"
lulc_path = r"Factor/LULC_2020.tif"

In [ ]:
# -------------------------------
# LULC 分类
# -------------------------------
lulc_classes = {
    1: "Cropland",
    2: "Forest",
    3: "Grassland",
    4: "Water Area",
    5: "Urban Residential Land",
    6: "Urban Commercial and Service Land",
    7: "Mixed-Use Land",
    8: "Other Urban Land",
    9: "Large-Scale Industrial Land",
    10: "Other Industrial Land",
    11: "Rural Settlement",
    12: "Transportation Land",
    13: "Unutilized Land"
}
valid_lulc_values = set(lulc_classes.keys())

In [ ]:
# -------------------------------
# 读取栅格
# -------------------------------
with rasterio.open(pv_change_path) as src_pv:
    pv_data = src_pv.read(1)

with rasterio.open(lulc_path) as src_lulc:
    lulc_data = src_lulc.read(1)


In [ ]:
# -------------------------------
# 自动裁剪到相同行列数
# -------------------------------
h = min(pv_data.shape[0], lulc_data.shape[0])
w = min(pv_data.shape[1], lulc_data.shape[1])

pv_data = pv_data[:h, :w]
lulc_data = lulc_data[:h, :w]

In [ ]:
# -------------------------------
# 清理 LULC
# -------------------------------
lulc_mask = np.isin(lulc_data, list(valid_lulc_values))
lulc_data_clean = np.where(lulc_mask, lulc_data, np.nan)

In [ ]:
# -------------------------------
# 统计函数
# -------------------------------
def compute_percentages(arr):
    valid = arr[~np.isnan(arr)]
    total = len(valid)
    stats = {}

    for k, v in lulc_classes.items():
        if total == 0:
            stats[v] = 0.0
        else:
            stats[v] = round(np.sum(valid == k) / total * 100, 2)

    return stats


In [ ]:
# -------------------------------
# 计算占比
# -------------------------------
overall_stats = compute_percentages(lulc_data_clean)
stats_2 = compute_percentages(lulc_data_clean[pv_data == 2])
stats_3 = compute_percentages(lulc_data_clean[pv_data == 3])
stats_4 = compute_percentages(lulc_data_clean[pv_data == 4])

In [ ]:
# -------------------------------
# 输出 Excel
# -------------------------------
df = pd.DataFrame({
    "LULC Class": list(lulc_classes.values()),
    "Overall (%)": list(overall_stats.values()),
    "PV Reduction (2) (%)": list(stats_2.values()),
    "PV Addition (3) (%)": list(stats_3.values()),
    "PV Persistence (4) (%)": list(stats_4.values())
})

df.to_excel("LULC_stats.xlsx", index=False)

print("处理完成并导出：LULC_stats.xlsx")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import squarify  # 用于Treemap
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 10

# ======================
# 1. 读取数据
# ======================
df = pd.read_excel("LULC_stats.xlsx")

classes = df["LULC Class"]
cols = ["Overall (%)", "PV Reduction (2) (%)", "PV Addition (3) (%)", "PV Persistence (4) (%)"]

x = np.arange(len(classes))


# ======================
# 2. 分组条形图（修正对齐）
# ======================
width = 0.2
plt.figure(figsize=(12,6))
for i, c in enumerate(cols):
    plt.bar(x + (i - 1.5)*width, df[c], width, label=c)

plt.xticks(x, classes, rotation=90)
plt.ylabel("Percentage")
plt.title("Grouped Bar Chart of Land Use Composition")
plt.legend()
plt.tight_layout()
plt.savefig("grouped_bar.png")
plt.close()


# ======================
# 3. 堆叠条形图（使用 x 保证一致）
# ======================
plt.figure(figsize=(10,6))
bottom = np.zeros(len(df))

for c in cols:
    plt.bar(x, df[c], bottom=bottom, label=c)
    bottom += df[c]

plt.xticks(x, classes, rotation=90)
plt.ylabel("Percentage")
plt.title("Stacked Bar Chart")
plt.legend()
plt.tight_layout()
plt.savefig("stacked_bar.png")
plt.close()


# ======================
# 4. 100% 堆叠条形图
# ======================
df_norm = df.copy()
df_norm[cols] = df_norm[cols].div(df_norm[cols].sum(axis=1), axis=0) * 100

plt.figure(figsize=(10,6))
bottom = np.zeros(len(df_norm))

for c in cols:
    plt.bar(x, df_norm[c], bottom=bottom, label=c)
    bottom += df_norm[c]

plt.xticks(x, classes, rotation=90)
plt.ylabel("Percentage (Normalized)")
plt.title("100% Stacked Bar Chart")
plt.legend()
plt.tight_layout()
plt.savefig("stacked_bar_100.png")
plt.close()


# ======================
# 5. Treemap 只基于 Overall(%)
# ======================
sizes = df["Overall (%)"]
labels = [f"{cl}\n{val:.2f}%" for cl, val in zip(classes, sizes)]

plt.figure(figsize=(12,7))
squarify.plot(sizes=sizes, label=labels, alpha=0.8)
plt.title("Treemap of Land Use Composition (Overall)")
plt.axis('off')
plt.tight_layout()
plt.savefig("treemap_overall.png")
plt.close()


# ======================
# 6. Donut Chart（环形图）
# ======================
fig, ax = plt.subplots(figsize=(6,6))
ax.pie(df["Overall (%)"], labels=classes, wedgeprops=dict(width=0.4))
plt.title("Donut Chart (Overall)")
plt.tight_layout()
plt.savefig("donut_overall.png")
plt.close()


# ======================
# 7. Polar Bar / Rose Chart（玫瑰图）
# ======================
theta = np.linspace(0.0, 2*np.pi, len(df), endpoint=False)
r = df["Overall (%)"]

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, polar=True)
bars = ax.bar(theta, r, width=2*np.pi/len(df), bottom=0)

ax.set_xticks(theta)
ax.set_xticklabels(classes, fontsize=8)
plt.title("Polar Bar Chart (Overall)")
plt.tight_layout()
plt.savefig("polar_bar.png")
plt.close()

print("All charts generated successfully!")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ======================
# 1. 读取数据
# ======================
df = pd.read_excel("LULC_stats.xlsx")
classes = df["LULC Class"]

# 计算占比差：区域占比 - 总体占比
diff_df = pd.DataFrame({
    'PV Reduction (2)': df['PV Reduction (2) (%)'] - df['Overall (%)'],
    'PV Addition (3)': df['PV Addition (3) (%)'] - df['Overall (%)'],
    'PV Persistence (4)': df['PV Persistence (4) (%)'] - df['Overall (%)']
}, index=classes)

# ======================
# 2. 偏移条形图
# ======================
plt.figure(figsize=(14,6))
x = np.arange(len(classes))
width = 0.25

plt.bar(x - width, diff_df['PV Reduction (2)'], width, label='PV Reduction', color='red')
plt.bar(x, diff_df['PV Addition (3)'], width, label='PV Addition', color='green')
plt.bar(x + width, diff_df['PV Persistence (4)'], width, label='PV Persistence', color='blue')

plt.xticks(x, classes, rotation=90)
plt.ylabel('Percentage Difference (%)')
plt.title('Deviation of Land Use Composition from Overall')
plt.legend()
plt.tight_layout()
plt.savefig("LULC_deviation_bar.png")
plt.show()

# ======================
# 3. 热力图
# ======================
plt.figure(figsize=(10,8))
sns.heatmap(diff_df, annot=True, cmap='RdBu_r', center=0, linewidths=0.5)
plt.title('Heatmap of Land Use Deviations from Overall')
plt.ylabel('LULC Class')
plt.xlabel('PV Change Type')
plt.tight_layout()
plt.savefig("LULC_deviation_heatmap.png")
plt.show()
